In [2]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time
import pymongo
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\skdab\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\skdab\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


# NASA Mars News

In [4]:
# URL of page to be scraped
url = 'https://redplanetscience.com'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [5]:
news_title = soup.body.find("div", class_="content_title").text.strip()
news_p = soup.body.find("div", class_="article_teaser_body").text.strip()
print(f"{news_title}: {news_p}")

Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth: Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight.


# JPL Mars Space Images-Featured Image

In [6]:
# Navigate to the page
url = 'https://spaceimages-mars.com/'
browser.visit(url)
html=browser.html
soup=bs(html,"html.parser")

In [7]:
featured_image = soup.body.find("img", class_="headerimage")["src"]
featured_image_url = url + featured_image
print(f"{featured_image_url}")


https://spaceimages-mars.com/image/featured/mars1.jpg


# Mars Facts

In [8]:
# Scrape Mars facts from https://galaxyfacts-mars.com
url = "https://galaxyfacts-mars.com/"
mars_facts = pd.read_html(url)
mars_facts[0]


,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
mars_facts_df = mars_facts[0]
mars_facts_df.columns = mars_facts_df.iloc[0]
mars_facts_df = mars_facts_df.drop(mars_facts[0].index[0])
mars_facts_df = mars_facts_df.set_index("Mars - Earth Comparison")
mars_facts_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [10]:
mars_facts_html = mars_facts_df.to_html()
mars_facts_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

# Mars Hemispheres

In [12]:
# Visit Mars Hemispheres website and take HTML code
base_url = "https://marshemispheres.com/"
browser.visit(base_url)
html = browser.html
soup = bs(html, "html.parser")

In [13]:
div_list = soup.body.find_all("div", class_="description")

In [14]:
sites = []

for div in div_list:
    site = div.find("a", class_="product-item")["href"]
    sites.append(site)

sites

['cerberus.html', 'schiaparelli.html', 'syrtis.html', 'valles.html']

In [15]:
hemisphere_image_urls = []

for site in sites:
    try:
        url = base_url + site
        browser.visit(url)
        html = browser.html
        soup = bs(html, "html.parser")
        img_url_ending = soup.body.find("img", class_="wide-image")["src"]
        img_url = base_url + img_url_ending
        title = soup.body.find("h2", class_="title").text.strip()[:-9]
        hemisphere_image_urls.append({"title": title, "img_url": img_url})
    except Exception as e:
        print(e)

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
